In [ ]:
import pandas as pd
from pyprojroot import here
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve

from colors import *

from nutils import bootstrap_auc

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))

auc_values = {}

for path in (here() / 'data/processed/matrices/prob').glob('*-lgbm-11-*.csv'):
    
    TARGET = path.stem.split('-')[0]
    MODEL = path.stem.split('-')[1]
    FS = path.stem.split('-')[2]
    HPO = path.stem.split('-')[3]
    FSE = path.stem.split('-')[4]

    if MODEL=='guess' or TARGET=='cri':
        continue
    
    pred_name = f'{TARGET}-{MODEL}-{FS}-{HPO}-{FSE}'
    true_name = f'{TARGET}'
    
    y_true = pd.read_csv(here() / f'data/processed/true_matrices/{true_name}.csv', index_col='Datetime')
    y_pred = pd.read_csv(here() / f'data/processed/matrices/prob/{pred_name}.csv', index_col='Datetime')

    df = pd.concat([y_true, y_pred], axis=1).dropna()
    y_true = df.iloc[:,0]
    y_pred = df.iloc[:,1]

    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    _auc = auc(fpr, tpr)

    lb, ub = bootstrap_auc(y_true, y_pred)
    
    ax.plot(fpr, tpr, 
            label=f"{TARGET.capitalize()} {_auc:.2f} ({lb:.2f}-{ub:.2f})", 
            color=c[TARGET])
    
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.legend(frameon=False);
    #ax.legend(frameon=False)

    ident = [0.0, 1.0]
    ax.plot(ident,ident, ls='--', c='k', zorder=0, lw=.7)

    ax.set_xlabel('FPR')
    ax.set_ylabel('TPR')

    auc_values[pred_name] = _auc

plt.savefig(here() / 'output/plots/auc.png', dpi=300, bbox_inches='tight')